In [ ]:
%pylab inline
import struct

# Read in data from binary waterfall file

In [ ]:
header_len = 8+26 # 2 unsigned int for center_freq and span: 8 bytes PLUS 26 bytes for datetime
nbin = 1024

with open("wf.bin", "rb") as fd:
    buff = fd.read()

length = len(buff[header_len:])
n_t = length / nbin

header = struct.unpack('2I26s', buff[:header_len])
data = struct.unpack('%dB'%length, buff[header_len:])

waterfall_array = np.reshape(np.array(data[:]), (n_t, nbin))
waterfall_array -= 255

avg_wf = np.mean(waterfall_array[:,:], axis=0)

center_frequency, span, rec_time = header
start_freq = center_frequency - span/2
stop_freq = center_frequency + span/2
print "Recording date:", rec_time

# Plot waterfall data

In [ ]:
figure(figsize=(40,8))
nticks = 11
xticks(np.linspace(0,nbin,nticks), np.linspace(start_freq, stop_freq, nticks))
pcolormesh(waterfall_array[:,1:], cmap=cm.jet , vmin=-100, vmax=-46)
title("Recording time: %s"%rec_time)
colorbar()

# Plot a few spectra

In [ ]:
figure(figsize=(20,8))
nticks = 11
xticks(np.linspace(0,nbin,nticks), np.linspace(start_freq, stop_freq, nticks))
for spectrum in waterfall_array[1:10]:
    plot(spectrum[1:])

# Plot signal levels distribution (instantaneous and average)

In [ ]:
figure(figsize=(20,8))
_=hist(waterfall_array[0,:], bins=40, normed=True)
xlim(-110,0)
xlabel("Signal level (dB)")
ylabel("Occurrences")
title("Instantaneous power level distribution")

figure(figsize=(20,8))
_=hist(avg_wf, bins=40, normed=True)
xlim(-110,0)
xlabel("Signal level (dB)")
ylabel("Occurrences")
title("Average power level distribution")


# Compute SNR for the whole band (average data)

In [ ]:
median = np.median(avg_wf)
perc95 = np.percentile(avg_wf, 95)

print "SNR estimation: median: %f dB, 95th perc.: %f dB, SNR: %f dB" % (median, perc95, perc95-median)